In [62]:
import pandas as pd 
import pymysql 
import pymysql.cursors
import numpy as np
from scipy.stats import chisquare
from scipy import stats
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols, logit, glm
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [63]:
excel_path = "C:\import\gihosung\palatibility_0126.xlsx"
pala_animal_fat = pd.read_excel( excel_path, sheet_name = "pala_animal_fat")
pala_dictype4857 = pd.read_excel( excel_path, sheet_name = "pala_dictype4857")
pala_protein = pd.read_excel( excel_path, sheet_name = "pala_protein")
product_list = pd.read_excel( excel_path, sheet_name = "product_list")
main_protein_count = pd.read_excel( excel_path, sheet_name = "main_protein_count")

In [64]:
main_protein_count_list = main_protein_count['product'].tolist()


In [65]:
result = []
for product_id in product_list['product'].tolist() :
    #####1.계유 어유 처리
    
    o_fish = int(pala_animal_fat[pala_animal_fat['product'] == int(product_id)]['fish_oil']) #지금 [0]을 붙이면 왜 안되는거지 >> 위 셀처럼 int를 붙여야겠따
    o_chicken = int(pala_animal_fat[pala_animal_fat['product'] == int(product_id)]['chicken_oil'])
    
    protein_list = pala_protein[pala_protein['product'] == int(product_id)]['protein_id'].tolist()
    main_protein_list = pala_protein[pala_protein['product'] == int(product_id)]['main_protein'].tolist()
    hydrolysis_list =pala_protein[pala_protein['product'] == int(product_id)]['hydrolysis'].tolist()
    main_protein_count_list = main_protein_count['product'].tolist()
    if  product_id in main_protein_count_list :
        main_protein_number = int(main_protein_count[main_protein_count['product']== int(product_id)]['main_protein_count']) 
   
    
    p_cow = 0 #A03
    p_chicken = 0 #B01
    fish_list = ['C0', 'C05', 'C06','C07','C08', 'C09', 'C1', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C27', 'C28', 'C29', 'C3',  'C32', 'C33','C34','C35','C36','C37','C38','C39','C4','C40','C41', 'C42', 'C43','C44']
    p_fish = 0 #C0, C05, C06,C07,C08, C09, C1, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19, C2, C20, C21, C22, C23, C24, C25, C27, C28, C29, C3,  C32, C33,C34,C35,C36,C37,C38,C39,C4,C40,C41, C42, C43,C44
    p_turkey = 0 #B02
    p_sheep = 0 #A04
    p_pig = 0 #A01
    p_duck = 0 #B03
    p_salmon = 0 #C26
    p_tuna = 0 #C31
    p_insect = 0 #G0, G00, G11, G12, G13, G14, G15, G16, G17, G18
    insect_list = ['G0', 'G00', 'G11', 'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18']
    
    
    ##### 2.주단백 + 가수분해단백질 처리
    for i,data in enumerate(protein_list) :
        if main_protein_number != 0 :
            #소_단백질원
            if  (data == 'A03' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_cow = 3/int(main_protein_number) + p_cow
                
            if (data == 'A03' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_cow = p_cow + 3
                
            if (data == 'A03' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_cow = 3/int(main_protein_number) + p_cow
                
            #닭_단백질원
            if  (data == 'B01' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_chicken = 3/main_protein_number + p_chicken
                
            if (data == 'B01' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_chicken = p_chicken + 3
                
            if (data == 'B01' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_chicken = 3/main_protein_number + p_chicken
                
            #생선_단백질원
            if  (data in fish_list) and main_protein_list[i] ==1 and hydrolysis_list[i] ==0 :
                p_fish = 3/main_protein_number + p_fish
                
            if (data in fish_list) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_fish = p_fish + 3
                
            if (data in fish_list) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_fish = 3/main_protein_number + p_fish
            
            #칠면조_단백질원
            if  (data == 'B02' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_turkey = 3/main_protein_number + p_turkey
                
            if (data == 'B02' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_turkey = p_turkey + 3
                
            if (data == 'B02' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_turkey = 3/main_protein_number + p_turkey
                
            #양_단백질원
            if  (data == 'A04' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_sheep = 3/main_protein_number + p_sheep
                
            if (data == 'A04' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_sheep = p_sheep + 3
                
            if (data == 'A04' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_sheep = 3/main_protein_number + p_sheep
                
            #돼지_단백질원
            if  (data == 'A01' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_pig = 3/main_protein_number + p_pig
                
            if (data == 'A01' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_pig = p_pig + 3
                
            if (data == 'A01' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_pig = 3/main_protein_number + p_pig
                
            #오리_단백질원
            if  (data == 'B03' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_duck = 3/main_protein_number + p_duck
                
            if (data == 'B03' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_duck = p_duck + 3
                
            if (data == 'B03' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_duck = 3/main_protein_number + p_duck
                
            #연어_단백질원
            if  (data == 'C26' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_salmon = 3/main_protein_number + p_salmon
                
            if (data == 'C26' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_salmon = p_salmon + 3
                
            if (data == 'C26' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_salmon = 3/main_protein_number + p_salmon
            
            #참치_단백질원
            if  (data == 'C31' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==0 :
                p_tuna = 3/main_protein_number + p_tuna
                
            if (data == 'C31' ) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_tuna = p_tuna + 3
                
            if (data == 'C31' ) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_tuna = 3/main_protein_number + p_tuna
            
            #곤충_단백질원
            if  (data in insect_list) and main_protein_list[i] ==1 and hydrolysis_list[i] ==0 :
                p_insect = 3/main_protein_number + p_insect
                
            if (data in insect_list) and main_protein_list[i] ==0 and hydrolysis_list[i] ==1 :
                p_insect = p_insect + 3
                
            if (data in insect_list) and main_protein_list[i] == 1 and hydrolysis_list[i] ==1 :
                p_insect = 3/main_protein_number + p_insect
    
    ### type 48,57 인 향미제 추가
    
    dic_type4857_list = pala_dictype4857[pala_dictype4857['product'] == int(product_id)]['protein'].tolist()
    for k in dic_type4857_list :
        if "닭" in k :
            p_chicken = p_chicken + 3
        if "돼지" in k :
            p_pig = p_pig +3
        if "생선" in k :
            p_fish = p_fish +3
        if "소" in k :
            p_cow = p_cow +3
        if "양" in k :
            p_sheep = p_sheep +3
        if "연어" in k :
            p_salmon = p_salmon +3
        if "오리" in k :
            p_duck = p_duck +3
        if "칠면조" in k :
            p_turkey = p_turkey +3
            
    result.append([product_id, p_cow, p_chicken, p_fish, p_turkey, p_sheep, p_pig, p_duck, p_salmon, p_tuna, p_insect, o_chicken, o_fish])
    

In [66]:
column = ['product_id', 'p_cow', 'p_chicken', 'p_fish', 'p_turkey', 'p_sheep', 'p_pig', 'p_duck', 'p_salmon', 'p_tuna', 'p_insect', 'o_chicken', 'o_fish']
final = pd.DataFrame.from_records( result , columns = column )



In [67]:
hydrolysis_add_0126 = pd.read_excel("C:\import\gihosung\hydrolysis_add_0126.xlsx")


In [68]:
df = hydrolysis_add_0126
for product, protein_id, 비고 in zip(df['product'],df['protein_id'],df['비고']) :
    if 비고 == 1:
        if protein_id == 'A01' :
            final.loc[final['product_id'] == product,'p_pig'] = final.loc[final['product_id'] == product,'p_pig'] + 3
        if protein_id == 'A03' :
             final.loc[final['product_id'] == product,'p_cow'] = final.loc[final['product_id'] == product,'p_cow'] + 3
        if protein_id == 'A04' :
             final.loc[final['product_id'] == product,'p_sheep'] = final.loc[final['product_id'] == product,'p_sheep'] + 3
        if protein_id == 'B01' :
            final.loc[final['product_id'] == product,'p_chicken'] = final.loc[final['product_id'] == product,'p_chicken'] + 3
        if protein_id == 'B03' :
             final.loc[final['product_id'] == product,'p_duck'] = final.loc[final['product_id'] == product,'p_duck'] + 3
        if protein_id == 'C26' :
             final.loc[final['product_id'] == product,'p_salmon'] = final.loc[final['product_id'] == product,'p_salmon'] + 3
        if protein_id == 'G13' :
             final.loc[final['product_id'] == product,'p_insect'] = final.loc[final['product_id'] == product,'p_insect'] + 3
            
new_final = final

In [54]:
new_final[new_final['product_id']==217]

,product_id,p_cow,p_chicken,p_fish,p_turkey,p_sheep,p_pig,p_duck,p_salmon,p_tuna,p_insect,o_chicken,o_fish
147,217,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1,1


In [61]:
new_final.to_excel("0126_2_score_default.xlsx")

In [38]:
final[final['product_id'] == 212]

,product_id,p_cow,p_chicken,p_fish,p_turkey,p_sheep,p_pig,p_duck,p_salmon,p_tuna,p_insect,o_chicken,o_fish
143,212,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
